- Can we calculate perplexity with a base of exp? much easier from the loss directly
- get batches in utils instead?
- down project activation (nope)
- what is the length of the continuation sentences

In [0]:
from google.colab import drive
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.ERROR)  # suppress some deprecation warnings
drive.mount('/gdrive')

In [0]:
# Tensorboard in Colab environment.
!pip install tensorboardcolab
from tensorboardcolab import *
tbc = TensorBoardColab()

In [0]:
# upload rest .py files to colab
# model.py, data_processing.py, utils.py
!rm *.py
from google.colab import files
files.upload()

In [0]:
# used to be able to rerun this cell
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

log_dir = "/tmp/tensorflow/logs"

# needed to avoid errors in this version of tensorflow
tf.flags.DEFINE_string('f', '', 'kernel')

tf.flags.DEFINE_string("log_dir", log_dir, "Summaries log directory")
tf.flags.DEFINE_integer("embedding_size", 100, "embedding size (default 100)")
tf.flags.DEFINE_integer("hidden_state_size", 512, "hidden state size (default 512)")
tf.flags.DEFINE_integer("batch_size", 64, "batch Size (default: 64)")
tf.flags.DEFINE_integer("max_vocabulary_size", 20000, "Maximum vocabulary size (default: 20000)")
tf.flags.DEFINE_integer("num_epochs", 10, "Number of training epochs (default: 10)")
tf.flags.DEFINE_integer("evaluate_every", 500, "Evaluate model on dev set after this many steps (default: 1000)")
tf.flags.DEFINE_integer("checkpoint_every", 5000, "Save model after this many steps (default: 5000)")
tf.flags.DEFINE_integer("num_checkpoints", 50, "Number of checkpoints to store (default: 50)")
tf.flags.DEFINE_integer("sentence_length", 30, "Maximum length of a sentence (default 30)")
tf.flags.DEFINE_boolean("load_embeddings", False, "Whether to use pretrained embeddings or not (default False)")
tf.flags.DEFINE_integer("down_project_size", None, 
                        "Down projection size. Should be used with a hidden_state_size of 1024 (default None)")

FLAGS = tf.app.flags.FLAGS

assert(FLAGS.down_project_size == 512 and FLAGS.hidden_state_size == 1024 or 
       FLAGS.down_project_size is None and FLAGS.hidden_state_size == 512)

In [5]:
print("\nCommand-line Arguments:")
for key in FLAGS.flag_values_dict():
    if key == 'f':
        continue
    print("{:<22}: {}".format(key.upper(), FLAGS[key].value))
print(" ")


Command-line Arguments:
LOG_DIR               : /tmp/tensorflow/logs
EMBEDDING_SIZE        : 100
HIDDEN_STATE_SIZE     : 512
BATCH_SIZE            : 64
MAX_VOCABULARY_SIZE   : 20000
NUM_EPOCHS            : 10
EVALUATE_EVERY        : 500
CHECKPOINT_EVERY      : 5000
NUM_CHECKPOINTS       : 50
SENTENCE_LENGTH       : 30
LOAD_EMBEDDINGS       : False
DOWN_PROJECT_SIZE     : None
 


In [6]:
from data_processing import DataProcessing

dataProcessing = DataProcessing(FLAGS.sentence_length, FLAGS.max_vocabulary_size)
dataProcessing.preprocess_dataset('/gdrive/My Drive/Colab Notebooks/NLU/data/', 'sentences.train', 'sentences.eval', 'sentences.continuation')

print(f'Vocabulary size\t\t\t\t{len(dataProcessing.vocab)}')
print(f'Number of train sentences is \t\t{len(dataProcessing.train_corpus)}')
print(f'Number of validation sentences is \t{len(dataProcessing.validation_corpus)}')
print(f'Number of continuation sentences is \t{len(dataProcessing.continuation_corpus)}')

Vocabulary size				20000
Number of train sentences is 		1969833
Number of validation sentences is 	9846
Number of continuation sentences is 	10000


In [0]:
from model import LSTMCell
from utils import train_batch, dev_step, load_embedding, continue_sentence
from data_processing import get_batches

In [0]:
with tf.Graph().as_default():
    with tf.Session() as session:
        # Create a variable to contain a counter for the global training step.
        global_step = tf.Variable(1, name='global_step', trainable=False)

        lstm = LSTMCell(FLAGS.embedding_size, FLAGS.hidden_state_size, FLAGS.sentence_length, 
                        FLAGS.max_vocabulary_size, down_project_size=FLAGS.down_project_size, 
                        pad_symbol=dataProcessing.vocab['<pad>'])

        if FLAGS.load_embeddings:
            load_embedding(session, dataProcessing.vocab, lstm.input_embeddings, 
                           '/gdrive/My Drive/Colab Notebooks/NLU/data/wordembeddings-dim100.word2vec',
                           FLAGS.embedding_size, len(dataProcessing.vocab))
            
        ####
        ## Set optimizer and crop all gradients to values [-5, 5]
        #### 
        with tf.name_scope('train'):
            optimizer = tf.train.AdamOptimizer()
            gvs = optimizer.compute_gradients(lstm.loss)
            capped_gvs = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gvs]
            train_step = optimizer.apply_gradients(capped_gvs, global_step=global_step)


        # Tensorboard
        train_writer = tbc.get_deep_writers("single_layer_model/train")
        train_writer.add_graph(session.graph)
        valid_writer = tbc.get_deep_writers("single_layer_model/valid")
        valid_writer.add_graph(session.graph)


        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        session.run(tf.global_variables_initializer())
        summaries_merged = tf.summary.merge(lstm.summaries)
        
        
        ####
        ## Create checkpoint directory
        #### 
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(FLAGS.log_dir, "runs", timestamp))
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
            
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")

        ####
        ## Start training for the specified epochs
        #### 
        for epoch in range(FLAGS.num_epochs):
            for sentences_batch in get_batches(dataProcessing.train_corpus, batch_size=FLAGS.batch_size):
                
                # run a single step
                train_batch(sentences_batch, lstm, train_step, global_step, session, summaries_merged, train_writer)
                
                current_step = tf.train.global_step(session, global_step)
                
                if current_step % FLAGS.evaluate_every == 0:
                    dev_step(get_batches(dataProcessing.validation_corpus, batch_size=FLAGS.batch_size), 
                             lstm, global_step, session, valid_writer, summaries_merged)
                    
                    generated_sentence = continue_sentence(dataProcessing.continuation_corpus[5], session, lstm, 
                                                           dataProcessing.inverse_vocab, dataProcessing.vocab['<eos>'])
                    print('Generated sentence #', generated_sentence, '# at step', current_step)
                
                    
                if current_step % FLAGS.checkpoint_every == 0:
                    path = saver.save(session, checkpoint_prefix, global_step=current_step)
                    print("Saved model checkpoint to {}".format(path))

            print('Done with epoch', epoch + 1)


        train_writer.flush()
        valid_writer.flush()

Evaluation average perplexity per across sentences is 275.453 at step 500
Generated sentence # when he was . # at step 500


In [0]:
# How to restore session

with tf.Graph().as_default():
    with tf.Session() as session:
        global_step = tf.Variable(1, name='global_step', trainable=False)
        lstm = LSTMCell(FLAGS.embedding_size, FLAGS.hidden_state_size, FLAGS.sentence_length, 
                        FLAGS.max_vocabulary_size, down_project_size=FLAGS.down_project_size, 
                        pad_symbol=dataProcessing.vocab['<pad>'])
        
        saver = tf.train.Saver()
        saver.restore(session, "/tmp/tensorflow/logs/runs/1554549194/checkpoints/model-5000")

In [0]:
!ls /tmp/tensorflow/logs/runs/1554549194/checkpoints/

In [0]:
from google.colab import files

# files.download('/tmp/tensorflow/logs/runs/1554499773/checkpoints/model-30000.data-00000-of-00001')
# files.download('/tmp/tensorflow/logs/runs/1554499773/checkpoints/model-30000.index')
# files.download('/tmp/tensorflow/logs/runs/1554499773/checkpoints/model-30000.meta')